## 1) 데이터 준비와 전처리

### Dataset

In [1]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings = ratings[['user_id', 'movie_id', 'count']]

In [5]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석해 봅시다.

### EDA

In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [11]:
# 가장 인기있는 영화 30개(인기순)
movie_data = pd.merge(ratings, movies)
movie_count = movie_data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

### Add ratings

In [12]:
my_favorite = ['Forrest Gump (1994)' , 'Back to the Future (1985)' ,'Toy Story (1995)' ,'Men in Black (1997)' ,'Jurassic Park (1993)']
favorite_movie_id = movies[movies['title'].isin(my_favorite)]
my_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': favorite_movie_id['movie_id'], 'count':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movie)
ratings.tail(10)

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,1,5
352,6041,356,5
476,6041,480,5
1250,6041,1270,5
1539,6041,1580,5


In [13]:
favorite_movie_id

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
352,356,Forrest Gump (1994),Comedy|Romance|War
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1250,1270,Back to the Future (1985),Comedy|Sci-Fi
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi


In [14]:
movie_data = pd.merge(ratings, movies, on='movie_id')
movie_data

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836478,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,Slaughterhouse (1987),Horror
836480,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


## 4) CSR matrix를 직접 만들어 봅시다.

### CSR matrix

In [15]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
0,6041,1,5
352,6041,356,5
476,6041,480,5
1250,6041,1270,5


In [16]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

In [17]:
num_user

6040

In [18]:
num_movie

3628

In [19]:
ratings.user_id

0          1
1          1
2          1
3          1
4          1
        ... 
0       6041
352     6041
476     6041
1250    6041
1539    6041
Name: user_id, Length: 836483, dtype: int64

In [20]:
ratings.movie_id

0       1193
1        661
2        914
3       3408
4       2355
        ... 
0          1
352      356
476      480
1250    1270
1539    1580
Name: movie_id, Length: 836483, dtype: int64

In [21]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

### Train

In [22]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [23]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [24]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [25]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [32]:
my_vector, forrestgump_vector = als_model.user_factors[6041], als_model.item_factors[356]

In [33]:
my_vector

array([-7.3723984e-01,  9.2900395e-01,  8.0492294e-01,  6.5497124e-01,
       -5.9860808e-01,  1.9622408e-01,  4.1941751e-02,  3.0651909e-01,
       -6.0419988e-02,  7.2956163e-01, -9.7903275e-01, -9.2431569e-01,
       -6.1701477e-01,  9.1576919e-02,  1.0504512e+00,  2.8616756e-01,
       -3.1830701e-01, -6.7333527e-02,  3.7989527e-01, -8.5492186e-02,
       -2.5254256e-01,  1.1072784e-01,  2.1849519e-01,  2.9884541e-01,
       -9.6226744e-02,  5.6716633e-01,  2.9750615e-01,  2.0831074e-01,
        5.9226352e-01,  3.8160917e-01, -8.2781595e-01, -6.5383571e-01,
       -5.9486455e-01,  9.1866836e-02, -3.5598907e-01, -7.5803745e-01,
       -4.9341351e-01,  2.0424366e-01,  9.9081881e-03,  3.1987649e-01,
       -6.0675800e-01, -4.0236115e-01,  4.8375145e-01, -7.6605424e-02,
       -1.3643359e-01, -4.7601494e-01, -2.0083070e-01,  7.0638835e-01,
       -1.5013461e-01,  5.5859011e-01, -3.6748943e-01,  5.8187169e-01,
       -7.1310826e-02, -9.5384777e-01,  1.5617148e-02,  1.5348533e+00,
      

In [34]:
forrestgump_vector

array([-0.01933125,  0.03021171,  0.07389245,  0.02813329, -0.02478398,
        0.04401866, -0.01845264,  0.00280822, -0.00226754,  0.01456546,
       -0.00175688, -0.03169024, -0.03243297, -0.00477321,  0.0226477 ,
       -0.00328994, -0.01488974,  0.02084849,  0.01887302,  0.00902637,
       -0.00850225, -0.01856042,  0.0188866 ,  0.01464758,  0.01001119,
        0.01671583,  0.02077429,  0.02011597, -0.0025981 ,  0.00676139,
        0.00170169, -0.00181487, -0.00411079, -0.00072514,  0.00793457,
       -0.00423099,  0.00603885,  0.01526021,  0.01358252,  0.01800299,
       -0.00163219, -0.03495573,  0.03357925, -0.0122302 ,  0.00136571,
        0.02473952,  0.01683808,  0.03239209, -0.00163543,  0.04966444,
        0.0191794 ,  0.0220244 ,  0.02368482, -0.04753416,  0.00474707,
        0.02653028, -0.00644963,  0.0091105 ,  0.0089324 , -0.00899012,
       -0.03015395,  0.03386762,  0.00361108,  0.0330761 ,  0.01727251,
       -0.01425582,  0.00024997,  0.0095741 , -0.00672049,  0.01

In [35]:
np.dot(my_vector, forrestgump_vector)

0.65771633

In [42]:
jumanji_vector = als_model.item_factors[2]
np.dot(my_vector, jumanji_vector)

-0.0042830072

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

### Similar Movie

In [37]:
favorite_movie = 'Forrest Gump (1994)'
movie_id = movies[movies['title']=='Forrest Gump (1994)']['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
similar_movie

[(356, 0.99999994),
 (1265, 0.5854424),
 (1784, 0.5377464),
 (597, 0.5344012),
 (587, 0.51558846),
 (539, 0.46092317),
 (39, 0.44995227),
 (2321, 0.43948358),
 (985, 0.42010814),
 (1316, 0.41193053),
 (357, 0.41193014),
 (1569, 0.40457532),
 (11, 0.4002461),
 (2671, 0.3971261),
 (402, 0.39326856)]

In [38]:
movies[movies['movie_id'].isin([s[0] for s in similar_movie])]

,movie_id,title,genre
10,11,"American President, The (1995)",Comedy|Drama|Romance
38,39,Clueless (1995),Comedy|Romance
352,356,Forrest Gump (1994),Comedy|Romance|War
353,357,Four Weddings and a Funeral (1994),Comedy|Romance
398,402,Open Season (1996),Comedy
535,539,Sleepless in Seattle (1993),Comedy|Romance
583,587,Ghost (1990),Comedy|Romance|Thriller
593,597,Pretty Woman (1990),Comedy|Romance
973,985,Small Wonders (1996),Documentary
1245,1265,Groundhog Day (1993),Comedy|Romance


## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

### Favorite Movie

In [39]:
user = 6041
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(3114, 0.42853066),
 (589, 0.41356826),
 (110, 0.3760215),
 (2858, 0.36432678),
 (1265, 0.35930994),
 (780, 0.30911505),
 (2571, 0.30240268),
 (2028, 0.29287314),
 (1210, 0.28779596),
 (2355, 0.28220087),
 (1721, 0.2681988),
 (2916, 0.25764668),
 (260, 0.25486517),
 (588, 0.2517958),
 (1097, 0.24469757),
 (1784, 0.23480879),
 (34, 0.23201492),
 (3175, 0.22989339),
 (2628, 0.22793591),
 (1196, 0.22509277)]

In [40]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])]

,movie_id,title,genre
33,34,Babe (1995),Children's|Comedy|Drama
108,110,Braveheart (1995),Action|Drama|War
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
770,780,Independence Day (ID4) (1996),Action|Sci-Fi|War
1081,1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1245,1265,Groundhog Day (1993),Comedy|Romance


### 느낀점(회고)



좋아하는 영화로 다음의 영화들을 선택하였습니다.
'Forrest Gump (1994)',
'Back to the Future (1985)',
'Toy Story (1995)',
'Men in Black (1997)',
'Jurassic Park (1993)'


Forrest Gump에 대한 선호도를 측정한 결과, 0.657 정도가 측정되었고, 좋아하는 영화 목록에 없는 jumanji에 대한 선호도를 측정한 결과, -0.004 정도가 측정되었습니다.


Forrest Gump와 유사한 영화를 추천받아봤는데, Forrest Gump의 장르인 Comedy|Romance|War 와 비슷한 장르의 영화를 추천받았습니다.


좋아하는 영화추천에서는 대부분의 장르가 Comedy이었습니다.초기에 목록에 넣은 영화들의 장르에 Comedy가 많이 포함되어 있어서 이런 결과를 얻은것 같습니다.
